In [2]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed

seed(1)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor 

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
df=pd.read_excel('new.xlsx')
df.head()

,Dew Point,Wind Speed,Pressure,Pressure Grad
0,4.8,2.47,1029.0,-8
1,8.2,7.42,1021.4,0
2,6.8,6.81,1021.8,11
3,3.6,3.94,1033.7,-1
4,6.1,3.33,1033.4,-11


In [4]:
df.isnull().sum()

Dew Point        0
Wind Speed       0
Pressure         0
Pressure Grad    0
dtype: int64

In [5]:
X = df.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y= df['Wind Speed']

In [6]:
X_norm=(X-X.min())/(X.max()-X.min())
X_norm

,Dew Point,Pressure,Pressure Grad
0,0.487805,0.752055,0.449275
1,0.606272,0.647945,0.565217
2,0.557491,0.653425,0.724638
3,0.445993,0.816438,0.550725
4,0.533101,0.812329,0.405797
...,...,...,...
1456,0.400697,0.160274,0.811594
1457,0.292683,0.394521,0.492754
1458,0.432056,0.338356,0.536232
1459,0.651568,0.321918,0.550725


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_norm, Y, test_size=0.2, random_state=0)

In [8]:
model = Sequential()
model.add(Dense(32, input_dim=x_train.shape[1], activation="sigmoid", kernel_initializer='normal'))
model.add(Dropout(0.2)) #dropping a few neurons for generalizing the model
model.add(Dense(1, activation="linear", kernel_initializer='normal'))
adam = Adam(learning_rate=1e-3, decay=1e-3)

# Compile model
model.compile(loss="mean_squared_error", optimizer='adam', metrics=['mse','mae'])

In [9]:
print('Fit model...')
filepath="/home/m-marouni/Documents/CE-901/Heathrow/best_weights"
checkpoint = ModelCheckpoint(filepath, monitor='val_mae', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_mae', patience=100, verbose=1, mode='min')
callbacks_list = [checkpoint, early_stopping]

log = model.fit(x_train, y_train,
          validation_split=0.40, batch_size=30, epochs=1000, shuffle=True, callbacks=callbacks_list)

Fit model...
Epoch 1/1000
24/24 [==============================] - 11s 18ms/step - loss: 51.5062 - mse: 51.5062 - mae: 6.5229 - val_loss: 47.3461 - val_mse: 47.3461 - val_mae: 6.1442

Epoch 00001: val_mae improved from inf to 6.14416, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 2/1000
24/24 [==============================] - 0s 3ms/step - loss: 46.7340 - mse: 46.7340 - mae: 6.1492 - val_loss: 42.3513 - val_mse: 42.3513 - val_mae: 5.7247

Epoch 00002: val_mae improved from 6.14416 to 5.72471, saving model to /home/m-marouni/Documents/CE-901/Heathrow/best_weights
INFO:tensorflow:Assets written to: /home/m-marouni/Documents/CE-901/Heathrow/best_weights/assets
Epoch 3/1000
24/24 [==============================] - 0s 3ms/step - loss: 40.3582 - mse: 40.3582 - mae: 5.6429 - val_loss: 37.6591 - val_mse: 37.6591 - val_mae: 5.3001

Epoch 00003: val_mae improved from 5

In [10]:
df.head()

,Dew Point,Wind Speed,Pressure,Pressure Grad
0,4.8,2.47,1029.0,-8
1,8.2,7.42,1021.4,0
2,6.8,6.81,1021.8,11
3,3.6,3.94,1033.7,-1
4,6.1,3.33,1033.4,-11


In [11]:
df_test=pd.read_excel('liverpool-Test-Data.xlsx')
df_test.head()

,Dew point,Wind Speed,Pressure,Pressure Gradient
0,16.6,5,1014,-2
1,16.6,4,1012,-6
2,18.0,4,1006,-5
3,17.0,5,1001,-3
4,17.0,8,998,-3


In [12]:
X2 = df_test.drop(['Wind Speed'], axis=1)
#Assign the Target column as the output 
Y2 = df_test['Wind Speed']

In [13]:
X2_norm=(X2-X2.min())/(X2.max()-X2.min())
X2_norm

,Dew point,Pressure,Pressure Gradient
0,0.000000,1.0000,1.00
1,0.000000,0.8750,0.00
2,1.000000,0.5000,0.25
3,0.285714,0.1875,0.75
4,0.285714,0.0000,0.75


In [14]:
new_predictions=model.predict(X2_norm)

In [15]:
new_predictions

array([[ 5.2819386],
       [ 4.890623 ],
       [ 6.1005926],
       [ 9.099541 ],
       [10.118674 ]], dtype=float32)

In [16]:
mean_absolute_error(Y2, new_predictions)

1.8982738494873046